In [1]:
!pip install ultralytics opencv-python-headless --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
from google.colab import files
uploaded = files.upload()


In [4]:
import os
print(os.listdir())  # should list your video


['.config', 'drive', 'sample_data']


In [5]:
import cv2
import numpy as np
from ultralytics import YOLO
from math import atan2, degrees


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
model_path = '/content/drive/MyDrive/cheating-detection-models/yolo11_cheating/weights/best.pt'
model = YOLO(model_path)


In [7]:
def estimate_head_pose(image, face_box):
    (x, y, w, h) = face_box
    h_img, w_img = image.shape[:2]

    # Define 2D image points (example points)
    image_points = np.array([
        (x + w // 2, y + int(h * 0.4)),   # Nose bridge
        (x + int(w * 0.3), y + int(h * 0.35)),  # Left eye
        (x + int(w * 0.7), y + int(h * 0.35)),  # Right eye
        (x + w // 2, y + int(h * 0.6)),   # Nose tip
        (x + int(w * 0.35), y + int(h * 0.8)),  # Left mouth
        (x + int(w * 0.65), y + int(h * 0.8))   # Right mouth
    ], dtype='double')

    # 3D model points
    model_points = np.array([
        (0.0, 0.0, 0.0),          # Nose bridge
        (-30.0, -30.0, -30.0),    # Left eye
        (30.0, -30.0, -30.0),     # Right eye
        (0.0, -60.0, -30.0),      # Nose tip
        (-20.0, -80.0, -30.0),    # Left mouth
        (20.0, -80.0, -30.0)      # Right mouth
    ])

    # Camera internals
    focal_length = w_img
    center = (w_img / 2, h_img / 2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype="double")

    dist_coeffs = np.zeros((4, 1))

    # Solve PnP
    success, rotation_vector, _ = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs)

    # Convert to angles
    rmat, _ = cv2.Rodrigues(rotation_vector)
    pose_mat = cv2.hconcat([rmat, np.zeros((3, 1))])
    _, _, _, _, _, _, eulerAngles = cv2.decomposeProjectionMatrix(pose_mat)
    pitch, yaw, roll = eulerAngles.flatten()

    # Classify direction
    direction = "Forward"
    if yaw > 20:
        direction = "Looking Right"
    elif yaw < -20:
        direction = "Looking Left"
    elif pitch > 20:
        direction = "Looking Down"
    elif pitch < -20:
        direction = "Looking Up"

    return direction


In [8]:
cap = cv2.VideoCapture('/content/drive/MyDrive/cheating_sample/cheating_sample_video.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = '/content/output_with_pose.mp4'
out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)[0]
    for box in results.boxes:
        cls = int(box.cls[0])
        label = model.names[cls]
        conf = box.conf[0].item()
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        box_w, box_h = x2 - x1, y2 - y1

        # Draw YOLO detection
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0) if label == 'normal' else (0, 0, 255), 2)
        cv2.putText(frame, f"{label} ({conf:.2f})", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Estimate head pose
        direction = estimate_head_pose(frame, (x1, y1, box_w, box_h))
        cv2.putText(frame, direction, (x1, y2 + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    out.write(frame)
    frame_count += 1


cap.release()
out.release()
print(f"Total frames written: {frame_count}")


Total frames written: 633


In [9]:
import os

if os.path.exists(output_path):
    print("✅ Video saved successfully:", output_path)
else:
    print("❌ Output video not found.")


✅ Video saved successfully: /content/output_with_pose.mp4


In [10]:
from google.colab import files
files.download(output_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>